# Weather Underground Hurricane Data

-----

## Processed Data Research

A notebook for researching the processed Weather Underground data from the ```src/process_data.py``` script.

In [ ]:
# Imports
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Variables
processed_dir = '../data/processed/'

# - Load Data
with open(os.path.join(processed_dir, 'region_data.pkl'), 'rb') as fin:
    region_df = pickle.load(fin)

with open(os.path.join(processed_dir, 'region_yearly_data.pkl'), 'rb') as fin:
    region_yearly_df = pickle.load(fin)

with open(os.path.join(processed_dir, 'storm_track_data.pkl'), 'rb') as fin:
    storm_track_dict = pickle.load(fin)


In [ ]:
# - Plot data by region
regions = ['North Atlantic', 'East Pacific', 'Western Pacific', 'Indian Ocean']

stats = ['Storms', 'Hurricanes', 'Deaths', 'Damage']
colors = ['orange', 'b', 'r', 'g']

fig, axs = plt.subplots(nrows=4, ncols=4, sharex=True, figsize=(12, 8))

i_col = 0
for region in regions:
    t_reg_df = region_df.loc[:, region]
    
    i_row = 0
    for statistic in stats:
        ax = axs[i_row][i_col]
        clr = colors[i_row]
        t_reg_df.loc[:, statistic].plot(ax=ax, color=clr)
        
        ax.set_yticklabels([])
        if i_col == 0:
            ax.set_ylabel(statistic)
    
        if i_row == 0:
            ax.set_title(region)
    
        i_row += 1
    
    i_col += 1

fig.savefig('../media/region_data_by_region_stat.png')
plt.show();